<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/TimeSeries/TimeSeries-2025/New/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%96%E2%80%AF8a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Практическая работа № 8a. Кросс-доменное обучение: трансфер- и мета-подходы к прогнозированию

## 1. Цель и задачи работы

**Цель работы** — формирование у обучающегося системного представления о методологии кросс-доменного машинного обучения, применяемой к задачам прогнозирования временных рядов, с акцентом на сравнительный анализ трансферного и метаобучения в условиях ограниченной доступности данных в целевом домене. Работа направлена на освоение практических навыков проектирования, адаптации и оценки моделей, способных переносить знания между семантически или статистически смежными доменами, а также обучаться «научиться прогнозировать» по малым фрагментам временных рядов.

**Основные задачи работы**:

- Использовать не менее **трёх временных рядов** из разных, но смежных доменов (например, энергопотребление в трёх регионах, продажи трёх категорий товаров, трафик на трёх маршрутах), каждый из которых:
  – содержит не менее 720 последовательных наблюдений;
  – прошёл предварительную обработку (стационарность подтверждена тестами ADF/KPSS, стабилизирована дисперсия, сформированы инженерные признаки: лаги, скользящие окна, циклические кодировки);
  – содержит внешние регрессоры (погода, праздники, индексы активности и т.п.);
  – строго разделён на исходный (source) и целевой (target) домены, при этом объём обучающих данных в target ограничен (≤100 наблюдений).

- Формализовать задачу прогнозирования как задачу обучения при **доменном сдвиге**: \( p_s(x, y) \neq p_t(x, y) \), с явным указанием типа сдвига (ковариатный, условный, временной тренд).

- Определить архитектуры и гиперпараметры для всех моделей и провести их обучение с соблюдением строгого разделения данных:
  – обучение на исходном домене (source);
  – адаптация или метаобучение с использованием ограниченной выборки из целевого домена (target);
  – оценка качества прогноза — исключительно на тестовой части целевого домена.

- Реализовать и сравнить следующие категории подходов:
  – **Трансферное обучение**:
    • Fine-tuning (дообучение головы или всей модели на target);
    • Feature-based transfer (заморозка энкодера, обучение только регрессора);
    • Adversarial domain adaptation (DANN, MMD-based выравнивание);
  – **Метаобучение**:
    • MAML (Model-Agnostic Meta-Learning);
    • Reptile;
    • Meta-трансформеры с общим backbone’ом и задачно-специфичными проекциями;
  – **Базовые и гибридные линии**:
    • Обучение «с нуля» на target (нижняя граница);
    • Multi-task learning на всех доменах совместно;
    • Аугментация target синтетическими данными из source (с использованием TimeGAN из Практической работы № 8).

- Реализовать **условную адаптацию**: прогнозирование в целевом домене при заданном режиме (например, «только летние месяцы» или «периоды высокой волатильности»).

- Сгенерировать прогнозы:
  – горизонт прогноза — не менее 30 шагов вперёд;
  – для каждого подхода — прогнозы для всех целевых доменов.

- Провести углублённую диагностику качества по трём критериям:
  – **Точность прогноза**: MAE, RMSE, MAPE, sMAPE на тестовой части target;
  – **Скорость адаптации**: количество точек из target, необходимых для достижения 90 % от upper bound качества;
  – **Устойчивость к переобучению**: дисперсия метрик при различных seed’ах и объёмах target.

- Оценить стабильность и эффективность подходов:
  – время дообучения / метаобновления;
  – время генерации прогноза;
  – устойчивость к осцилляции loss и коллапсу адаптации.

- Проанализировать интерпретируемость репрезентаций:
  – визуализация скрытых состояний (t-SNE, UMAP) до и после адаптации;
  – вычисление расстояния между доменами в пространстве признаков (MMD, CORAL);
  – анализ attention-мап в трансформерах — выявление доменно-инвариантных паттернов.

- Исследовать практические сценарии:
  – **Быстрый старт**: развёртывание прогноза для нового продукта/региона с двухнедельной историей;
  – **Имитация сдвига**: адаптация модели к постпандемическому поведению на основе доковидных данных;
  – **Персонализация**: прогнозирование индивидуального спроса на основе общего опыта по группе клиентов.

- Разработать **модульный кросс-доменный фреймворк** с единым интерфейсом:
  – `fit(source_data, target_data_few_shot=None)`;
  – `adapt(target_data_few_shot)`;
  – `predict(horizon)`.

- Обеспечить воспроизводимость и открытость результатов:
  – опубликовать код, конфигурации, прогнозы и аналитический отчёт в публичном репозитории;
  – использовать открытые лицензии: MIT (код), CC BY 4.0 (данные и прогнозы).

## 2. Теоретические предпосылки

Кросс-доменное прогнозирование решает фундаментальную проблему несоответствия между распределением обучающих и тестовых данных — явления, особенно острого для временных рядов из-за нестационарности, структурных разрывов и эволюции внешних факторов. Традиционные модели, обученные «с нуля» на малом объёме целевых данных, страдают от высокой дисперсии и нестабильности.

**Трансферное обучение** преодолевает это ограничение за счёт переноса полезных репрезентаций из богатого исходного домена. Подходы варьируются от простого fine-tuning до сложных методов выравнивания распределений (например, через минимизацию максимальной средней несогласованности — MMD). Ключевая гипотеза: существует доменно-инвариантное подпространство, в котором зависимости \( y = f(x) \) остаются стабильными.

**Метаобучение** переформулирует задачу: вместо обучения одной модели на одном ряде, модель учится «научиться прогнозировать» на множестве задач, каждая из которых — короткий фрагмент временного ряда из отдельного домена. MAML и Reptile обеспечивают быструю адаптацию к новым условиям за 1–5 градиентных шагов, что критично при дефиците данных.

Современные архитектуры — **Temporal Fusion Transformer**, **Informer**, **TimesNet** — благодаря вниманию и иерархической обработке, демонстрируют высокую переносимость репрезентаций. Однако эффективность трансфера зависит не только от архитектуры, но и от степени семантической близости доменов и стратегии адаптации.

Важно подчеркнуть: цель кросс-доменного обучения — не устранение различий между доменами, а **максимизация полезности перенесённых знаний** при минимальной дообучаемости. Модель считается успешной, если достигает сопоставимого качества с upper bound при использовании ≤10 % данных целевого домена.

## 3. Порядок выполнения работы

### 3.1. Подбор и подготовка данных

Задача: Обеспечить реалистичные условия доменного сдвига.  
Требования:
– Использовать временные ряды, обработанные в Практической работе № 1, и дополнить их двумя новыми рядами из смежных доменов;
– Каждый ряд должен содержать ≥720 наблюдений, ≥5 признаков, внешние регрессоры;
– Для каждого ряда — выделить source (первые 80 %) и target (последние 20 %, из которых используется ≤100 точек для адаптации);
– Подтвердить статистическое несовпадение доменов (KS-тест, сравнение ACF).

### 3.2. Формализация задачи и предобработка

Задача: Корректно сформулировать задачу кросс-доменного прогнозирования.  
Требования:
– Определить тип доменного сдвига (ковариатный, временной тренд, seasonal shift);
– Нормализация признаков — только по source-части;
– Кодирование временных признаков — циклическое (sin/cos);
– Для метаобучения — формирование задач как коротких окон (например, 60 точек → прогноз 30).

### 3.3. Определение архитектур и обучение моделей

Задача: Привести каждую модель к рабочей конфигурации.  
Требования:
– Использовать реализации из библиотек `pytorch`, `tsai`, `darts`, или воспроизведение по статьям (MAML, DANN);
– Гиперпараметры по умолчанию или из литературы:
  • TFT: hidden_size=32, attention_heads=4;
  • MAML: inner_lr=0.01, meta_lr=0.001, k_shots=10;
  • DANN: градиентный инвертор с λ=1.0;
– Обучение на source, адаптация — только на выделенных ≤100 точках target;
– Фиксация: время обучения, стабильность loss, частота переобучения.

### 3.4. Генерация прогнозов

Задача: Получить сопоставимые прогнозы всех подходов.  
Требования:
– Горизонт прогноза: 30 шагов;
– Для каждого подхода — прогнозы по всем целевым доменам;
– Реализация условной адаптации: например, `adapt(condition={'season': 'summer'})`.

### 3.5. Построение и настройка моделей

Задача: Обеспечить единообразный интерфейс.  
Требования:
– Все модели реализуют методы:
  • `fit(source_data, target_data_few_shot=None)`;
  • `adapt(target_data_few_shot)`;
  • `predict(horizon)`;
– Для MAML и DANN — кастомная реализация на PyTorch при отсутствии open-source версии.

### 3.6. Диагностика качества прогнозирования

Задача: Объективно оценить эффективность переноса.  
Требования:
– **Точность**: MAE, RMSE, MAPE на тестовой части target;
– **Скорость адаптации**: кривые качества в зависимости от объёма target (10, 30, 50, 100 точек);
– **Устойчивость**: стандартное отклонение метрик по 5 запускам с разными seed’ами;
– **Анализ переобучения**: сравнение train/test loss при адаптации.

### 3.7. Оценка качества и сравнительный анализ

Задача: Ранжировать подходы по комплексному критерию.  
Требования:
– Составление сравнительных таблиц:
  • Подход × Домен × MAE × Скорость адаптации × Время обучения;
– Ранжирование по взвешенной оценке:
  • Точность (50 %),
  • Скорость адаптации (30 %),
  • Вычислительная эффективность (20 %).

### 3.8. Анализ интерпретируемости и репрезентаций

Задача: Оценить качество выравнивания доменов.  
Требования:
– Визуализация скрытых состояний (t-SNE) до и после адаптации;
– Вычисление MMD между source и target в пространстве признаков;
– Для трансформеров — attention-мапы: стабильны ли они между доменами?

### 3.9. Анализ вычислительной эффективности

Задача: Оценить практическую применимость.  
Требования:
– Фиксация времени:
  • Обучение на source (минуты),
  • Адаптация на target (секунды),
  • Генерация прогноза (мс);
– Оценка памяти (memory_profiler);
– Классификация: низкая / средняя / высокая сложность.

### 3.10. Практические сценарии применения

Задача: Продемонстрировать ценность подходов в реальных задачах.  
Требования:
– **Быстрый старт**: прогноз спроса на новый товар с 2-недельной историей → сравнение F1/MAE с обучением с нуля;
– **Имитация сдвига**: адаптация модели энергопотребления к аномальной жаре → анализ отклонения от базового сценария;
– **Персонализация**: метаобучение по 10 клиентам → прогноз для 11-го с 5 наблюдениями.

### 3.11. Публикация результатов исследования

Задача: Обеспечить открытость и воспроизводимость.  
Требования:
– В репозитории:
  • Код фреймворка;
  • Конфигурации моделей;
  • Прогнозы в формате CSV/Parquet;
  • Сравнительные таблицы и визуализации;
– Лицензии: MIT (код), CC BY 4.0 (данные и прогнозы);
– README с инструкцией по запуску и воспроизведению экспериментов.

## 4. Дополнительные исследовательские задания

- Сравнение эффективности **MAML** и **fine-tuning** при экстремально малом target (≤10 точек);
- Анализ утечки информации: может ли модель восстановить исходные паттерны source’а из target’а?
- Генерация прогнозов с **жёсткими ограничениями**: «спрос > 100 при температуре > 30°C»;
- Перенос обучения между **несмежными доменами** (например, трафик → продажи) — оценка качества;
- Исследование зависимости качества от **степени доменного сдвига** (низкий / средний / высокий MMD);
- Оценка **энергоэффективности**: сравнение CO₂-следа MAML и DANN при адаптации.

## 5. Требования к отчёту

Отчёт оформляется в соответствии с **ГОСТ 7.32–2017** и должен содержать:

- **Введение** — постановка задачи, актуальность кросс-доменного прогнозирования, обзор литературы, преемственность с Практическими работами № 1 и № 8;
- **Методология** — описание данных, архитектур, метрик (точность, скорость адаптации, устойчивость), стратегий адаптации;
- **Результаты экспериментов** — сравнительные таблицы, графики качества, визуализации репрезентаций, примеры прогнозов;
- **Обсуждение** — анализ компромиссов между точностью, скоростью и сложностью; рекомендации по выбору подхода;
- **Заключение** — итоговые выводы, практические рекомендации, ограничения исследования;
- **Список использованных источников** — по ГОСТ Р 7.0.5–2008;
- **Приложения** — фрагменты кода, скриншоты, таблицы гиперпараметров, примеры прогнозов.

Отчёт сопровождается ссылкой на публичный репозиторий с кодом и результатами.

## 6. Критерии оценивания

| Оценка | Критерии |
|--------|--------|
| **Отлично** | Реализованы ≥4 подхода (включая трансфер и метаобучение); оценка по трём критериям (точность, скорость, устойчивость); условная адаптация; практические сценарии; модульный код; публикация; отчёт по ГОСТ. |
| **Хорошо** | Реализованы 2–3 подхода; есть сравнительные таблицы и базовая диагностика; отсутствует лишь отдельный элемент (например, анализ репрезентаций). |
| **Удовлетворительно** | Только fine-tuning и обучение с нуля; есть прогнозы и простые метрики; нет углублённой оценки адаптации. |
| **Неудовлетворительно** | Прогноз без оценки качества; нет разделения на source/target; отсутствует сравнительный анализ; отчёт не представлен. |
